Building AE using Pytorch

Let's start building a very simple autoencoder for the MNIST dataset using Pytorch. The MNIST dataset is eidely used for becnhmark dataset in machine learning and computer vision. It consists of a collection of 28x28 grayscale images of handwritten digits (0-9). The dataset is divided into a training set with 60.000 images and a test set with 10.000 images. It's often employed to evaluate the perfomance of various neural network architectures and algorithms for digit recognition tasks.

First of all, we will import libraries like torch, numpy, and matplotlib.

In [2]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import time
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from torch.utils.data import sampler
import numpy as np
import matplotlib.colors as mcolors

Then, we'll set some hyperparameter like learning rate, batch size, number of epochs and number of classes

In [3]:
#Hyperparameters
RANDOM_SEED = 42
LEARNING_RATE = 0.0005
BATCH_SIZE = 256
NUM_EPOCHS = 20
NUM_CLASSES = 10

Here we're downloading the MNIST dataset and setting the training and testing data loader with batch size of 256.

In [8]:
def get_dataloaders_mnist(batch_size, num_workers=0,
                          train_transforms=None, test_transforms=None):
    if train_transforms is None:
        train_transforms = transforms.ToTensor()
    if test_transforms is None:
        test_transforms = transforms.ToTensor()

    train_dataset = datasets.MNIST(root='data',
                                   train=True,
                                   transform=train_transforms, download=True)
    valid_dataset = datasets.MNIST(root='data',
                                   train=True,
                                   transform=test_transforms)
    test_dataset = datasets.MNIST(root='data',
                                  train=False,
                                  transform=test_transforms)
    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=batch_size,
                              num_workers=num_workers,
                              shuffle=True)
    valid_loader = DataLoader(dataset=valid_dataset,
                              batch_size=batch_size,
                              num_workers=num_workers,
                              shuffle=True)
    test_loader  = DataLoader(dataset=test_dataset,
                              batch_size=batch_size,
                              num_workers=num_workers,
                              shuffle=False)
    
    return train_loader,valid_loader,  test_loader

train_loader, valid_loader, test_loader = get_dataloaders_mnist(batch_size=BATCH_SIZE, num_workers=2)

Let's check the image batch dimensions and image label dimensions

In [9]:
#Checking the dataset
print('Training Set:\n')
for images,labels, in train_loader:
    print('Image batch dimensions:', images.size())
    print('Image label dimensions:', labels.size())
    print(labels[:10])
    break

Training Set:

Image batch dimensions: torch.Size([256, 1, 28, 28])
Image label dimensions: torch.Size([256])
tensor([8, 2, 8, 3, 4, 4, 1, 9, 2, 1])


Now, we're going to develop the model. The encoder uses four convolution layers with leaky relu as activation. The decoder has four transpose convolution layers and a trimming layer to get the generated image in (28, 28) dimension,

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, stride=(1,1), kernel_size=(3, 3), padding=1),
            nn.LeakyReLU(0.01),
            nn.Conv2d(32, 64, stride=(2, 2), kernel_size=(3, 3), padding=1),
            nn.LeakyReLU(0.01),
            nn.Conv2d(64, 64, stride=(2, 2), kernel_size=(3, 3), padding=1),
            nn.LeakyReLU(0.01),
            nn.Conv2d(64, 64, stride=(1, 1), kernel_size=(3, 3), padding=1),
            nn.Flatten(),
            nn.Linear(3136, 2)
        )

        self.decoder = nn.Sequential(
            torch.nn.Linear(2, 3136),
            torch.reshape(-1, 64, 7, 7),
            nn.ConvTranspose2d(64, 64, stride=(1, 1),
                               kernel_size=(3, 3), padding=1),
            nn.LeakyReLU(0.01),
            nn.ConvTranspose2d(64, 64, stride=(2, 2),
                               kernel_size=(3, 3), padding=1),
            nn.LeakyReLU(0.01),
            nn.ConvTranspose2d(64, 32, stride=(2, 2),
                               kernel_size=(3, 3), padding=0),
            nn.LeakyReLU(0.01),
            nn.ConvTranspose2d(32, 1, stride=(1, 1),
                               kernel_size=(3, 3), padding=0),
                               torch.trim


        )
    
    